In [1]:
import pandas as pd
import numpy as np

We'll start by reading in our fraud dataset and looking at the column names:

In [2]:
df = pd.read_csv("fraud.csv")
df.columns

Index(['timestamp', 'label', 'user_id', 'amount', 'merchant_id', 'trans_type',
       'foreign'],
      dtype='object')

# Transaction type distribution

In [3]:
pt = pd.pivot_table(df[["label", "trans_type", "timestamp"]], 
                    index=["label", "trans_type"], aggfunc=len)

pt.columns = ['count']
gdf = pd.DataFrame(pt.to_records())

gdf['total'] = gdf.groupby('label')['count'].transform(np.sum)
gdf['pctage'] = gdf['count'] / gdf['total']

gdf

,label,trans_type,count,total,pctage
0,fraud,chip_and_pin,21007,412839,0.050884
1,fraud,contactless,20678,412839,0.050087
2,fraud,manual,144390,412839,0.349749
3,fraud,online,206163,412839,0.499379
4,fraud,swipe,20601,412839,0.049901
5,legitimate,chip_and_pin,4507746,22533292,0.200048
6,legitimate,contactless,5631831,22533292,0.249934
7,legitimate,manual,1128292,22533292,0.050072
8,legitimate,online,7886722,22533292,0.350003
9,legitimate,swipe,3378701,22533292,0.149943


In [4]:
import altair as alt

alt.Chart(gdf).mark_bar().encode(
    alt.Y('pctage:Q', axis=alt.Axis(format='.0%')), column='trans_type', x="label", color='label'
)

alt.Chart(...)

# Foreign transaction distribution

In [5]:
pt = pd.pivot_table(df[["label", "foreign", "timestamp"]], 
                    index=["label", "foreign"], aggfunc=len)

pt.columns = ['count']
gdf = pd.DataFrame(pt.to_records())

gdf['total'] = gdf.groupby('label')['count'].transform(np.sum)
gdf['pctage'] = gdf['count'] / gdf['total']

gdf

alt.Chart(gdf).mark_bar().encode(
    alt.Y('pctage:Q', axis=alt.Axis(format='.0%')), column='foreign', x="label", color='label'
)

alt.Chart(...)

# Transaction amount distribution

In [6]:
%%time
qs = df[['label','amount']].groupby('label').quantile(q=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99])
qs

CPU times: user 1min 20s, sys: 2.06 s, total: 1min 22s
Wall time: 1min 30s


amount
label                    
fraud      0.01    5.3500
           0.05    6.7400
           0.10    8.9800
           0.25   14.7100
           0.50   22.6100
           0.75   28.1700
           0.90   35.3300
           0.95   40.6900
           0.99   53.0400
legitimate 0.01    3.3100
           0.05    4.6900
           0.10    6.7500
           0.25   12.4500
           0.50   20.0100
           0.75   28.0200
           0.90   46.0700
           0.95   91.8800
           0.99  496.4009

In [7]:
qdf = pd.DataFrame(qs.to_records())
alt.Chart(qdf).mark_line(interpolate="monotone").encode(
    alt.Y("amount", axis=alt.Axis(title='transaction amounts (log scale)'), scale=alt.Scale(type='log')), 
    alt.X("level_0", axis=alt.Axis(title='cumulative distribution'), scale=alt.Scale(type='linear')), 
    color="label"
)

alt.Chart(...)

# Interarrival times

In [8]:
trans_times = df[['timestamp', 'label', 'user_id']].sort_values(['user_id', 'timestamp'])
trans_times['date'] = pd.to_datetime(trans_times['timestamp'], unit='s')

In [9]:
interarrivals = trans_times.set_index(['user_id', 'label', 'date']).diff().reset_index()
interarrivals.columns = ['user_id', 'label', 'date', 'interarrival']
# interarrivals = interarrivals[interarrivals['interarrival'] > 0]

In [10]:
fraudsamp = interarrivals[(interarrivals['label'] == 'fraud')].copy()
legitsamp = interarrivals.loc[(interarrivals['label'] == 'legitimate'), :].sample(len(fraudsamp))


In [11]:
fraudsamp['irank'] = fraudsamp['interarrival'].rank(pct=True, method="dense")
legitsamp['irank'] = legitsamp['interarrival'].rank(pct=True, method="dense")
qdf = pd.concat([fraudsamp.groupby(['label', 'interarrival', 'irank']).size(), legitsamp.groupby(['label', 'interarrival', 'irank']).size()])
qdf = pd.DataFrame(pd.DataFrame(qdf).to_records())
qdf = qdf[qdf['interarrival'] > 0]

In [12]:
alt.Chart(qdf.sample(5000)).mark_line().interactive().encode(
    alt.Y("interarrival", axis=alt.Axis(title='interarrival time'), scale=alt.Scale(type='log')), 
    alt.X("irank", axis=alt.Axis(title='cumulative distribution'), scale=alt.Scale(type='linear')), 
    color="label"
)

alt.Chart(...)